In [ ]:
import pandas as pd
from pyspark import SparkContext
#sc = SparkContext.getOrCreate()
#sqlContext = SQLContext(sc)
spark

In [ ]:
%run ../ssb_sparktools/editing/editing.py

In [ ]:
%run ../ssb_sparktools/quality/quality.py

In [ ]:
%run ../ssb_sparktools/processing/processing.py

In [ ]:
skattRaw = spark.read.path('/produkt/skatt/person/temp/tverrsnitt_skattemelding')

In [ ]:
#skattRaw = spark.read.path('/produkt/skatt/person/temp/tverrsnitt_skattemelding1000rader')

In [ ]:
skattRaw.count()

In [ ]:
skatt_dict = unpack_parquet(skattRaw, rootdf=True, rootvar=True)

In [ ]:
#Kjører missing_df for alle skatteobjekter og samler resultatet i en dictionary
skatt_missing = {}
for k in skatt_dict.keys():
    skatt_missing[k] = missing_df(skatt_dict[k])

In [ ]:
#Eksempel på resultat fra missing_df
skatt_missing['rootdf']

In [ ]:
#Enda et eksempel på resultat fra missing_df
skatt_missing['fondskonto']

In [ ]:
#Pandas variant
#Kjører korrigering på bolske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_bool, missing_boolcount = missing_correction_bool(skatt_dict['fondskonto'])
missing_boolcount

In [ ]:
#Pandas variant
#Eksempel før korrigering
skatt_dict['fondskonto'][['identifikator', 'overfoertFraBarn']].toPandas()

In [ ]:
#Pandas variant
#Eksempel etter korrrigering
df_corrected_bool[['identifikator', 'overfoertFraBarn']].toPandas()

In [ ]:
#Pandas variant
#Eksempel før korrigering der overfoertFraBarn skal unntas korrigering
skatt_dict['konto'].select('identifikator', 'erBoligsparekontoIAnnenEOeSStat', 'overfoertFraBarn').toPandas()

In [ ]:
#Pandas variant
#Kjører korrigering på bolske verdier på et gitt datasett med unntak for 1 variabel. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_bool, missing_boolcount = missing_correction_bool(skatt_dict['konto'], exception_for=['overfoertFraBarn'])

In [ ]:
#Pandas variant
#Eksempel etter korrigering der overfoertFraBarn skal unntas korrigering
df_corrected_bool.select('identifikator', 'erBoligsparekontoIAnnenEOeSStat', 'overfoertFraBarn').toPandas()

In [ ]:
#Pandas variant
#Kjører korrigering på numeriske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_number, missing_numbcount = missing_correction_number(skatt_dict['fondskonto'])
missing_numbcount

In [ ]:
#Pandas variant
#Eksempel før korrigering
skatt_dict['fondskonto']\
                [['identifikator', 'skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel','skattepliktigTapVedRealisasjonAvOgUttakFraRentedel',\
                 'verdiFoerVerdsettingsrabattForAksjeOgAksjefond', 'skattefrittUttak', 'benyttetSkjermingsfradrag']]\
                .toPandas()

In [ ]:
#Pandas variant
#Eksempel etter korrigering
df_corrected_number\
                [['identifikator', 'skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel','skattepliktigTapVedRealisasjonAvOgUttakFraRentedel',\
                 'verdiFoerVerdsettingsrabattForAksjeOgAksjefond', 'skattefrittUttak', 'benyttetSkjermingsfradrag']]\
                .toPandas()

In [ ]:
#Pandas variant
#Kjører korrigering på numeriske verdier på et gitt datasett der skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel er unntatt korrigering. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_number, missing_numbcount = missing_correction_number(skatt_dict['fondskonto'], exception_for=['skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel'])

In [ ]:
#Pandas variant
#Eksempel etter korrigering der skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel er unntatt korrigering
df_corrected_number\
                [['identifikator', 'skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel','skattepliktigTapVedRealisasjonAvOgUttakFraRentedel',\
                 'verdiFoerVerdsettingsrabattForAksjeOgAksjefond', 'skattefrittUttak', 'benyttetSkjermingsfradrag']]\
                .toPandas()

In [ ]:
%%time
#Pandas variant
#Eksempel som kjører korrigering av numeriske og bolske verdier for alle skatteobjekt og samler loggene i et eget datasett
corrected_missing_log = pd.DataFrame(columns=['dataframe', 'datatype', 'variabel', 'korrigeringer'])

for k in skatt_dict.keys():
    
    df, miss_bool = missing_correction_bool(skatt_dict[k], df_name=k)
    corrected_missing_log = corrected_missing_log.append(miss_bool)
    df, miss_number = missing_correction_number(df, df_name=k)
    corrected_missing_log = corrected_missing_log.append(miss_number)

corrected_missing_log = corrected_missing_log.reset_index(drop=True)

In [ ]:
#Pandas variant
#Log av datesett
corrected_missing_log

In [ ]:
filtercorrect = corrected_missing_log['dataframe']=='aksjeIAksjonaerregisteret'
corrected_missing_log[filtercorrect]

In [ ]:
#Spark variant
df_corrected_bool, missing_boolcount = spark_missing_correction_bool(skatt_dict['fondskonto']) 

In [ ]:
df_corrected_bool[['identifikator', 'overfoertFraBarn']].toPandas()

In [ ]:
#Spark variant
#missing_correction_bool
#Kjører korrigering på bolske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_bool, missing_boolcount = spark_missing_correction_bool(skatt_dict['fondskonto'])
missing_boolcount.toPandas()

In [ ]:
df_corrected_bool[['identifikator', 'overfoertFraBarn']].toPandas()

In [ ]:
#spark_missing_correction_bool
#Kjører korrigering på bolske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_bool, missing_boolcount = spark_missing_correction_bool(skatt_dict['fondskonto'])
missing_boolcount.toPandas()

In [ ]:
df_corrected_bool[['identifikator', 'overfoertFraBarn']].toPandas()

In [ ]:
#spark_missing_correction_number
#Kjører korrigering på numeriske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_number, missing_numbcount = spark_missing_correction_number(skatt_dict['fondskonto'])
missing_numbcount.toPandas()

In [ ]:
#Eksempel etter korrigering
df_corrected_number\
                [['identifikator', 'skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel','skattepliktigTapVedRealisasjonAvOgUttakFraRentedel',\
                 'verdiFoerVerdsettingsrabattForAksjeOgAksjefond', 'skattefrittUttak', 'benyttetSkjermingsfradrag']]\
                .toPandas()

In [ ]:
#missing_correction_number
#Kjører korrigering på numeriske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_number, missing_numbcount = spark_missing_correction_number(skatt_dict['fondskonto'])
missing_numbcount.toPandas()

In [ ]:
#Eksempel etter korrigering
df_corrected_number\
                [['identifikator', 'skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel','skattepliktigTapVedRealisasjonAvOgUttakFraRentedel',\
                 'verdiFoerVerdsettingsrabattForAksjeOgAksjefond', 'skattefrittUttak', 'benyttetSkjermingsfradrag']]\
                .toPandas()

In [ ]:
%%time
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
missing_variabler = [StructField('dataframe', StringType(), False),\
                   StructField('variabel', StringType(), True),\
                   StructField('datatype', StringType(), True),\
                   StructField('korrigeringer', IntegerType(), False)]
missing_schema = StructType(missing_variabler)
df_corrections = sqlContext.createDataFrame(sc.emptyRDD(), missing_schema)

skatt_test={}

missing_corrections = []
for k in skatt_dict.keys():
    
    skatt_test[k], miss_bool = spark_missing_correction_bool(skatt_dict[k], df_name=k)
    if len(miss_bool.head(1))>0:
        df_corrections= df_corrections.union(miss_bool)        
    skatt_test[k], miss_number = spark_missing_correction_number(skatt_dict[k], df_name=k)
    if len(miss_number.head(1))>0:
        df_corrections= df_corrections.union(miss_number)        


In [ ]:
df_corrections.toPandas()